<a href="https://colab.research.google.com/github/lelascarnevali/py-Analise_Carteira/blob/main/Desvios_Normalidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 19.1 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import yfinance as yf

In [3]:
def Drawdown(serie_retorno: pd.Series):
  retornos_acc = (1+serie_retorno).cumprod()-1
  picos_retornos = retornos_acc.cummax()
  drawdown = ((1+retornos_acc)-(1+picos_retornos))/(1+picos_retornos)
  return pd.DataFrame({"Retornos_Acc": retornos_acc, 
                         "Picos_Retornos": picos_retornos, 
                         "Drawdown": drawdown})

In [4]:
yf.pdr_override()
tickers = "^BVSP ^GSPC GC=F USDBRL=X"
carteira = yf.download(tickers, interval='1mo', start="2008-01-01")["Close"]
carteira.columns = ["OURO", "DOLAR", "IBOV", "S&P500"]
carteira = carteira.dropna()
carteira["OURO_BRL"]=carteira["OURO"]*carteira["DOLAR"]
carteira["S&P500_BRL"]=carteira["S&P500"]*carteira["DOLAR"]

[*********************100%***********************]  4 of 4 completed


#Assimetria

Intuitivamente, uma inclinação negativa significa que você obtém mais retornos negativos do que esperaria se os retornos fossem distribuídos como a distribuição normal.

Outra maneira de pensar sobre isso é se esses retornos fossem normalmente distribuídos, a média e a mediana seriam muito próximas.

No entanto, se eles são assimétricos negativamente, o valor esperado, ou seja, a média é menor que a mediana. Se eles são positivamente assimétricos, o valor esperado (novamente, a média) é maior que a mediana.

In [5]:
pd.concat([carteira.mean(), carteira.median(), carteira.mean()>carteira.median()], axis=1)

,0,1,2
OURO,1343.658399,1293.000000,True
DOLAR,2.914915,2.408800,True
IBOV,68018.490078,61546.000000,True
S&P500,2049.527960,1932.229980,True
OURO_BRL,4071.102832,3425.133536,True
S&P500_BRL,6962.025666,4819.290793,True



Agora, vamos desenvolver o código para calcular a assimetria de uma série de números.

Lembre-se que a assimetria é dada por:

$$ S(R) = \frac{E[ (R-E(R))^3 ]}{\sigma_R^3} $$

In [6]:
def assimetria(r):

    # Para definir o desvio padrão da população, parametrizar dof=0
    sigma_r = r.std(ddof=0)
    exp = ((r - r.mean())**3).mean()
    return exp/sigma_r**3

In [7]:
assimetria(carteira).sort_values()

OURO          0.311207
DOLAR         0.797547
S&P500        0.951291
IBOV          1.074283
S&P500_BRL    1.421848
OURO_BRL      1.490639
dtype: float64

In [8]:
import scipy.stats
scipy.stats.skew(carteira)

array([0.31120651, 0.79754661, 1.07428275, 0.95129078, 1.49063868,
       1.42184839])

#Curtose

Intuitivamente, a curtose mede a "gordura" dos extremos da distribuição. A distribuição normal tem uma curtose de 3 e, portanto, se a curtose de seus retornos for menor que 3, ela tende a ter caudas mais finas, e se a curtose for maior que 3, a distribuição terá caudas mais gordas.

A curtose é dada por:

$$ K(R) = \frac{E[ (R-E(R))^4 ]}{\sigma_R^4} $$


Isso é muito semelhante à assimetria, então podemos apenas copiá-la e editá-la para calcular a 4ª em vez da 3ª potência (como foi o caso da assimetria).

In [10]:
def curtose(r):

    # Para definir o desvio padrão da população, parametrizar dof=0
    sigma_r = r.std(ddof=0)
    exp = ((r - r.mean())**4).mean()
    return exp/sigma_r**4

In [12]:
curtose(carteira).sort_values()

OURO          2.478601
DOLAR         2.588569
S&P500        3.355015
IBOV          3.400416
S&P500_BRL    4.255608
OURO_BRL      4.376108
dtype: float64

In [13]:
scipy.stats.kurtosis(carteira)

array([-0.521399  , -0.41143069,  0.40041578,  0.35501518,  1.37610751,
        1.255608  ])

Observe que esses números são todos menores em 3 em relação ao número que calculamos. Isso porque, como dissemos acima, a curtose esperada de uma série de números normalmente distribuídos é 3, e scipy.stats está retornando o Excesso de Curtose.


In [14]:
scipy.stats.kurtosis(carteira)+3

array([2.478601  , 2.58856931, 3.40041578, 3.35501518, 4.37610751,
       4.255608  ])